In [30]:
import sys
import os
import pandas as pd
import numpy as np
import biosignalsnotebooks as bsnb
import biosignalsnotebooks.signal_samples as bsnb_ss
import re
import pickle
import scipy as sp
import datetime

In [31]:
df = pd.read_csv('/Users/htr365/no_icloud/quantified_self_all/data/survey_data/raw_survey_data.csv')

In [32]:
names = ['timestamp','person']  + [f'cis_{i}' for i in range(1, 21)] \
+ ['hooper_muscles_sore','extra_muscles_tired','hooper_sleep_quality','hooper_fatigue_level','hooper_stress_level']  \
+ ['sleep_hours','coffee_count','physical_activity','step_count'] \
+ ['f_illness','f_changes','f_day_description','f_measurement_comments','f_other']  \
+ ['alcohol','fatigue_expected','X','muscles_sore_expected','muscles_tired_expected'] 

In [33]:
df.columns = names
df.head()

,timestamp,person,cis_1,cis_2,cis_3,cis_4,cis_5,cis_6,cis_7,cis_8,...,f_illness,f_changes,f_day_description,f_measurement_comments,f_other,alcohol,fatigue_expected,X,muscles_sore_expected,muscles_tired_expected
0,24/06/2024 23:40:05,Johanna,Disagree,Strongly agree,Disagree,Strongly agree,Strongly agree,Strongly agree,Somewhat agree,Somewhat agree,...,NaN,NaN,measurement + work from home ~1h\n20min cycle ...,"- first measurement I did without button, prot...",this was a bit of a test day,NaN,NaN,NaN,NaN,NaN
1,25/06/2024 13:52:02,Amanda,Somewhat agree,Neither agree or disagree,Somewhat disagree,Somewhat disagree,Somewhat agree,Somewhat agree,Neither agree or disagree,Somewhat disagree,...,"No, sore muscles and inflamed hand.",No,"Travelled to Brisbane 2 hours, install show 6 ...",Did first measurement twice as I wasn't on the...,Only 29 days to go!!!!,NaN,NaN,NaN,NaN,NaN
2,25/06/2024 22:33:22,Johanna,Disagree,Agree,Disagree,Neither agree or disagree,Somewhat agree,Somewhat agree,Neither agree or disagree,Agree,...,no,NaN,- 20min yoga stretching\n- 20min cycling to wo...,"pedometer showed 10500 steps, phone showed 140...",NaN,no alcohol,NaN,NaN,NaN,NaN
3,26/06/2024 13:51:24,Amanda,Agree,Somewhat disagree,Agree,Agree,Somewhat disagree,Somewhat disagree,Somewhat agree,Disagree,...,"Yes, dizzy, fatigued sore muscles",Not really different from yesterday,Install exhibition,Could not get open signals to work in PM,NaN,no alcohol,NaN,NaN,NaN,NaN
4,27/06/2024 01:17:04,Johanna,Disagree,Somewhat disagree,Disagree,Disagree,Somewhat agree,Somewhat agree,Somewhat agree,Agree,...,NaN,NaN,"cycled to work 20min\nworked 2.5 hours, lunch ...",Wasn't able to conduct a measurement in the ev...,NaN,no alcohol,NaN,NaN,NaN,NaN


In [34]:
# drop unnecessary column
df = df.drop('X',axis=1)

In [35]:
# convert to datetime and create date variable
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date

/var/folders/m2/yy15wc4d3zqdtsvvvggmkn780000gp/T/ipykernel_45405/1856565900.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['timestamp'] = pd.to_datetime(df['timestamp'])


In [36]:
# adjust for measurement errors
# Amanda, 17th of July: This survey is for the 16.07. Forgot to press SUBMIT 
df.loc[(df['date']==datetime.date(2024,7,17))* (df['person']=='Amanda'),'date'] = datetime.date(2024,7,16)

# Amanda, 18th of July: THIS IS FOR 17th. NOT 18th
df.loc[(df['timestamp']==datetime.datetime(2024,7,18,12,20,48))* (df['person']=='Amanda'),'date'] = datetime.date(2024,7,17)

# Amanda 21st of July
df.loc[(df['timestamp']==datetime.datetime(2024,7,21,11,1,52))* (df['person']=='Amanda'),'date']= datetime.date(2024,7,20)

# Amnda 24th of July
df.loc[(df['timestamp']==datetime.datetime(2024,7,24,5,44,48))* (df['person']=='Amanda'),'date']= datetime.date(2024,7,23)

# Amanda 31st of July
df.loc[(df['timestamp']==datetime.datetime(2024,7,31,7,19,13))* (df['person']=='Amanda'),'date']= datetime.date(2024,7,30)

# Amanda 3rd of August
df.loc[(df['timestamp']==datetime.datetime(2024,8,3,7,7,44))* (df['person']=='Amanda'),'date']= datetime.date(2024,8,2)

# Amanda 4th of August
df.loc[(df['timestamp']==datetime.datetime(2024,8,4,20,14,27))* (df['person']=='Amanda'),'date']= datetime.date(2024,8,3)

# Amanda 5th of August
df.loc[(df['timestamp']==datetime.datetime(2024,8,5,22,43,50))* (df['person']=='Amanda'),'date']= datetime.date(2024,8,4)

# Amanda 6th of August
df.loc[(df['timestamp']==datetime.datetime(2024,8,8,7,35,43))* (df['person']=='Amanda'),'date']= datetime.date(2024,8,6)
# Amanda 7th of August
df.loc[(df['timestamp']==datetime.datetime(2024,8,8,7,39,58))* (df['person']=='Amanda'),'date']= datetime.date(2024,8,7)

# Amanda 9th of August
df.loc[(df['timestamp']==datetime.datetime(2024,8,9,17,56,16))* (df['person']=='Amanda'),'date']= datetime.date(2024,8,8)

# Amanda 10th of August
df.loc[(df['timestamp']==datetime.datetime(2024,8,10,22,5,14))* (df['person']=='Amanda'),'date']= datetime.date(2024,8,9)

# Amanda 12th of August
df.loc[(df['timestamp']==datetime.datetime(2024,8,12,21,1,37))* (df['person']=='Amanda'),'date']= datetime.date(2024,8,11)


# Amanda 13th of August
df.loc[(df['timestamp']==datetime.datetime(2024,8,13,7,53,24))* (df['person']=='Amanda'),'date']= datetime.date(2024,8,12)


df.loc[(df['timestamp']==datetime.datetime(2024,8,14,22,30,5))* (df['person']=='Amanda'),'date']= datetime.date(2024,8,13)

# correct for Johanna often taken the survey after midnight
df['date'] = np.where((df['person']=='Johanna')*(df['timestamp'].dt.hour<20),df['date']-datetime.timedelta(days=1),df['date'])


In [37]:
# convert cis  items + hooper items + extra question to numeric scale

agree_likert_scale = {'strongly disagree': 1,
                'disagree': 2,
                'somewhat disagree': 3,
                'neither agree or disagree': 4,
                'somewhat agree': 5,
                'agree': 6,
                'strongly agree': 7}

agree_likert_scale_reverse = {'strongly disagree': 7,
                'disagree': 6,
                'somewhat disagree': 5,
                'neither agree or disagree': 4,
                'somewhat agree': 3,
                'agree': 2,
                'strongly agree': 1}

quality_likert_scale = {'very poor': 1,
                'poor': 2,
                'below average': 3,
                'average': 4,
                'above average': 5,
                'good': 6,
                'excellent': 7}

intensity_likert_scale = {'very low': 1,
                'low': 2,
                'below average': 3,
                'average': 4,
                'above average': 5,
                'high': 6,
                'very high': 7}

# fetch columns to change
items_agree = ['cis_1', 'cis_2','cis_4','cis_5', 'cis_7','cis_8','cis_9','cis_11','cis_14','cis_15','cis_16'] + ['hooper_muscles_sore','extra_muscles_tired','muscles_sore_expected','muscles_tired_expected']
items_agree_reverse = ['cis_3', 'cis_6','cis_10','cis_12', 'cis_13','cis_17','cis_18','cis_19','cis_20']

#'cis_5','cis_6','cis_7','cis_8','cis_11','cis_12','cis_15','cis_20']

items_quality = ['hooper_sleep_quality']
items_intensity = ['hooper_fatigue_level','hooper_stress_level','fatigue_expected']
# convert to numeric
df[items_agree] = df[items_agree].applymap(lambda x: x.lower() if isinstance(x, str) else x).replace(agree_likert_scale).apply(pd.to_numeric, errors='coerce')
df[items_agree_reverse] = df[items_agree_reverse].applymap(lambda x: x.lower() if isinstance(x, str) else x).replace(agree_likert_scale_reverse).apply(pd.to_numeric, errors='coerce')
df[items_quality] = df[items_quality].applymap(lambda x: x.lower() if isinstance(x, str) else x).replace(quality_likert_scale).apply(pd.to_numeric, errors='coerce')
df[items_intensity] = df[items_intensity].applymap(lambda x: x.lower() if isinstance(x, str) else x).replace(intensity_likert_scale).apply(pd.to_numeric, errors='coerce')

In [38]:
# subscales of CIS
# Subjective feeling of fatigue items 1, 4, 6, 9, 12, 14, 16, 20
df['cis_subjective_fatigue'] = df[[f'cis_{i}' for i in [1,4,6,9,12,14,16,20]]].mean(axis=1)

# Concentration items 3, 8, 11, 13, 19
df['cis_concentration'] = df[[f'cis_{i}' for i in [3,8,11,13,19]]].mean(axis=1)

# Motivation items 2, 5, 15, 18
df['cis_motivation'] = df[[f'cis_{i}' for i in [2,5,15,18]]].mean(axis=1)

# Physical activity items 7, 10, 17
df['cis_physical_activity'] = df[[f'cis_{i}' for i in [7,10,17]]].mean(axis=1)

In [39]:
# map alcohol consumption to numeric values
alcohol_dict = {'no alcohol': 0,
            'one light alcoholic drink (e.g. beer, wine)': 1,
            'two light alcoholic drinks or one strong alcoholic drink (e.g. cocktail, shot)': 2,
             'more than three alcoholic drinks':3
            }
df['alcohol'] = df['alcohol'].map(alcohol_dict).fillna(0)

In [40]:
# convert sleep time to minutes
df['sleep_time'] = pd.to_datetime(df['sleep_hours']).dt.hour*60 +pd.to_datetime(df['sleep_hours']).dt.minute

/var/folders/m2/yy15wc4d3zqdtsvvvggmkn780000gp/T/ipykernel_45405/3856294496.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sleep_time'] = pd.to_datetime(df['sleep_hours']).dt.hour*60 +pd.to_datetime(df['sleep_hours']).dt.minute
/var/folders/m2/yy15wc4d3zqdtsvvvggmkn780000gp/T/ipykernel_45405/3856294496.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sleep_time'] = pd.to_datetime(df['sleep_hours']).dt.hour*60 +pd.to_datetime(df['sleep_hours']).dt.minute


In [41]:
def split_physical_activity(cell):
    if isinstance(cell, str):
        list_activities = cell.split(',')
        for i in range(len(list_activities)):
            list_activities[i] = list_activities[i].strip()
        return list_activities

# convert physical activites from string list to actual list
df['physical_activity_list'] = df['physical_activity'].apply(split_physical_activity)

In [42]:
activities_list = []
for i in range(len(df['physical_activity_list'])):
    if df.loc[i, 'physical_activity_list'] is not None:
        activities_list += df.loc[i, 'physical_activity_list']

activities_list = pd.Series(activities_list)


In [43]:
activities_list[activities_list=='Spinning Class'] = 'High Intensive Interval Training'
activities_list[activities_list=='Boxing Class'] = 'High Intensive Interval Training'
activities_list[activities_list=='8 hours of manual labour (smashing rocks)'] = 'Manual Labour'
activities_list[activities_list=='N/A manual labour'] = 'Manual Labour'
activities_list[activities_list=='Biketrip ~40km']='Cycling for 30min or longer'
activities_list[activities_list=='Hiking ~7h'] = 'Hiking'

In [44]:
# count ocurrence of activities
activities_counts = activities_list.value_counts()
# keep only those occurring more than once
activities_list = activities_counts[activities_counts>1].index

In [45]:
activities_counts

Walking for 30min or longer                                     40
Cycling for 30min or longer                                     13
Weight Lifting                                                   7
Ball Sports                                                      7
Running                                                          7
High Intensive Interval Training                                 6
Swimming for 30min or longer                                     3
                                                                 3
Yoga                                                             2
Manual Labour                                                    2
Hiking                                                           2
Performance art course - walked around non stop in Salzburg.     1
Upper Body Workout                                               1
Full Body Workout with Weights                                   1
N/a                                                           

In [46]:
activities_list.unique()

Index(['Walking for 30min or longer', 'Cycling for 30min or longer',
       'Weight Lifting', 'Ball Sports', 'Running',
       'High Intensive Interval Training', 'Swimming for 30min or longer', '',
       'Yoga', 'Manual Labour', 'Hiking'],
      dtype='object')

In [47]:
def activity_in_list(row, activity):
    if row is not None:
        if activity in row:
            return 1
        else:
            0
    else:
        0
# create one column for each activity listed more than once and put 1 if it happened 0 otherwise
for activity in activities_list:
    df[activity] = df['physical_activity_list'].apply(lambda x: activity_in_list(x, activity)).fillna(0)


In [48]:
df = df.rename(columns= {'Walking for 30min or longer': 'walking',
'Cycling for 30min or longer': 'cycling',
'Ball Sports': 'ball_sports',
'Weight Lifting': 'weight_lifting',
'Running':'running',
'Yoga': 'yoga',
'High Intensive Interval Training':'HITT',
'Manual Labour': 'manual_labour',
'Hiking':'hiking'})
df = df.drop('',axis=1)

In [49]:
# create lagged fatigue
df['cis_subjective_fatigue_lead1']  = df.groupby('person')['cis_subjective_fatigue'].shift(-1)
df['extra_muscles_tired_lead1']  = df.groupby('person')['extra_muscles_tired'].shift(-1)
df['hooper_muscles_sore_lead1']  = df.groupby('person')['hooper_muscles_sore'].shift(-1)

In [50]:
# get max number of activities on a given day
#df[['walking','cycling','ball_sports','weight_lifting','running','yoga','spinning']].sum(axis=1).max()

In [51]:
df.to_csv('/Users/htr365/no_icloud/quantified_self_all/data/survey_data/pre_processed_survey.csv')

In [52]:
df[df['person']=='Amanda']['date']

1     2024-06-25
3     2024-06-26
16    2024-07-08
18    2024-07-09
20    2024-07-10
22    2024-07-11
24    2024-07-12
26    2024-07-13
28    2024-07-14
30    2024-07-15
33    2024-07-16
35    2024-07-17
36    2024-07-18
38    2024-07-19
41    2024-07-20
42    2024-07-21
44    2024-07-22
47    2024-07-23
48    2024-07-24
51    2024-07-26
53    2024-07-27
55    2024-07-28
57    2024-07-29
58    2024-07-30
59    2024-07-31
60    2024-08-02
61    2024-08-03
62    2024-08-04
63    2024-08-05
64    2024-08-06
65    2024-08-07
66    2024-08-08
67    2024-08-09
68    2024-08-10
69    2024-08-11
70    2024-08-12
71    2024-08-13
72    2024-08-14
Name: date, dtype: object

In [53]:
df[['person','date','cis_1', 'cis_2', 'cis_3', 'cis_4', 'cis_5',
       'cis_6', 'cis_7', 'cis_8', 'cis_9', 'cis_10', 'cis_11', 'cis_12',
       'cis_13', 'cis_14', 'cis_15', 'cis_16', 'cis_17', 'cis_18', 'cis_19',
       'cis_20', 'hooper_muscles_sore', 'extra_muscles_tired',
       'hooper_sleep_quality', 'hooper_fatigue_level', 'hooper_stress_level',
       'sleep_hours', 'coffee_count', 'physical_activity', 'step_count','alcohol', 'fatigue_expected', 'muscles_sore_expected',
       'muscles_tired_expected', 'cis_subjective_fatigue', 'cis_concentration',
       'cis_motivation', 'cis_physical_activity', 'sleep_time','walking', 'cycling', 'ball_sports',
       'weight_lifting', 'running', 'yoga', 'HITT','manual_labour','hiking', 'cis_subjective_fatigue_lead1', 'hooper_muscles_sore_lead1','extra_muscles_tired_lead1']].to_csv('/Users/htr365/no_icloud/quantified_self_all/data/survey_data/pre_processed_survey_features.csv')

In [54]:
3327*12

39924